# Info 370 HW 08

Name: Ella Kim   

*Citations are hyperlinked*

## Is COMPAS fair? 
#### 1

In [23]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

#load in data
df_raw = pd.read_csv('/home/jovyan/lost+found/INFO_370/compas-score-data.csv.bz2', sep="\t")

#sanity checks
# print # cols
cols = df_raw.shape[1]
print("Number of columns: " + str(cols))
# print # rows
rows = df_raw.shape[0]
print("Number of rows: " + str(rows))
print(df_raw.head(5)) #sanity check
# make df of total count of nulls in each col & print results
df_null = df_raw.isnull().sum() # there are no null values

Number of columns: 8
Number of rows: 6172
   age c_charge_degree              race          age_cat   sex  priors_count  \
0   69               F             Other  Greater than 45  Male             0   
1   34               F  African-American          25 - 45  Male             0   
2   24               F  African-American     Less than 25  Male             4   
3   44               M             Other          25 - 45  Male             0   
4   41               F         Caucasian          25 - 45  Male            14   

   decile_score  two_year_recid  
0             1               0  
1             3               1  
2             4               1  
3             1               0  
4             6               1  


#### 2

In [24]:
# filter raw df for only African American and Caucasian races
df_compas = df_raw[(df_raw.race == "African-American") | (df_raw.race == "Caucasian")]

#### 3

In [25]:
# make a copy form to avoid warnings
df_compas = df_compas.copy()

(df_compas.decile_score <= 0).any().any() # no invalid values in column

# make new column with catagory type of risk level with inequalities, value is inclusive to right bin
df_compas["risk_cat"] = pd.cut(df_compas.decile_score,
                      bins = [-np.inf, 5, np.inf],
                      labels = ["low", "high"],
                      right=False)

# df_compas.risk_cat.tolist() # sanity check

#### 4

In [26]:
# first filter df for only binary 1 for recidivism and get total count
df_recidivism = df_compas[df_compas.two_year_recid == 1]
n_total_recid = df_recidivism.shape[0]

# A: compute recidivism rate vs risk
# get count of each
n_recid_low = df_recidivism[df_recidivism.risk_cat == "low"].shape[0]
n_recid_high = df_recidivism[df_recidivism.risk_cat == "high"].shape[0]
# compute and print rate
print("Recidivism rate for low risk: " + str(n_recid_low/n_total_recid))
print("Recidivism rate for high risk: " + str(n_recid_high/n_total_recid))

# B: compute recidivism rate vs race
# get count of each
n_recid_afram = df_recidivism[df_recidivism.race == "African-American"].shape[0]
n_recid_caucas = df_recidivism[df_recidivism.race == "Caucasian"].shape[0]
# compute and print rate
print("\nRecidivism rate for African American: " + str(n_recid_afram/n_total_recid))
print("Recidivism rate for Causaisan: " + str(n_recid_caucas/n_total_recid))

Recidivism rate for low risk: 0.35481272654047524
Recidivism rate for high risk: 0.6451872734595248

Recidivism rate for African American: 0.6689488521949255
Recidivism rate for Causaisan: 0.3310511478050745


#### 5

In [27]:
# make confusion matrix, first is row, 2nd is column 
cm = pd.crosstab(df_compas.two_year_recid, df_compas.risk_cat)
print(cm) # print matrix

# calculate different totals
total = cm.sum().sum()
[non_recid, recid] = cm.sum(axis = 1)
# assign cells in matrix to variable names I understand
tp = cm.iloc[1,1]
tn = cm.iloc[0,0]
fn = cm.iloc[1,0]
fp = cm.iloc[0,1]

# print results/answers for A,R,P,F and false rates
print()
print(str(round((tp+tn)/total * 100, 2)) + "% of individuals were accurately calssified by COMPAS (accuracy).\n")
print(str(round((tp/(recid)*100), 2)) + "% of recidivism were correctly classified as high-risk (recall), or mirror image: ")
print(str(round((tp/(tp+fp)* 100), 2)) + "% of high risk individuals were correcly classified as recidivism (precision), or the mean of the two: ")
print(str(round(2/((1/(tp/(recid)*100))+ (1/(tp/(tp+fp)* 100))), 2)) + "% average of high risk individuals and recidivism were correctly classified (F-score).\n")

print(str(round(((fn/recid)*100), 2)) + "% of recidivism were falsly classified as low-risk, or mirror image: ")
print(str(round(((fn/(tn+fn))*100), 2)) + "% of low-risk individuals were falsly classified as recidivism.\n")
print(str(round(((fp/non_recid)*100), 2)) + "% of no-recidivism were falsly classified as high-risk, or mirror image: ")
print(str(round(((fp/(tp+fp))*100), 2)) + "% of high-risk individuals were falsly classified as no-recidivism.")

risk_cat         low  high
two_year_recid            
0               1872   923
1                881  1602

65.82% of individuals were accurately calssified by COMPAS (accuracy).

64.52% of recidivism were correctly classified as high-risk (recall), or mirror image: 
63.45% of high risk individuals were correcly classified as recidivism (precision), or the mean of the two: 
63.98% average of high risk individuals and recidivism were correctly classified (F-score).

35.48% of recidivism were falsly classified as low-risk, or mirror image: 
32.0% of low-risk individuals were falsly classified as recidivism.

33.02% of no-recidivism were falsly classified as high-risk, or mirror image: 
36.55% of high-risk individuals were falsly classified as no-recidivism.


#### 6

I would not feel comfortable at all with a judge to use COMPAS, as the error in both predicted and actual are all over 30%, which means in the actual US prison population, of the hundreds of thousands of people who are released each year, over a third of them are inaccurately classified. Before this step's compuations, I was already considering 1% error bad enough as, in proportion to population, that would mean ~1,000 individuals are inaccurately classified, which is a large number, so I would never consider over 30% error ever tolerable in any situation. As human judges also are not perfect and would have their own biases even if only implicit/unconsious, so I am rethinking my 1% tolerance as a maybe impossible rate to achieve at this time. I beleive I would be more flexible after seeing COMPAS catagorize, but I also do not want to change my fundamentals just because of a algorithm that makes the situation worse and dulls my opinions towards misclassification. I would hope that someday a error as low or much lower than 1% would become possible.

#### 7

In [28]:
# now repeat above for race vs recidivism
cm_race = pd.crosstab(df_compas.two_year_recid, df_compas.race)
print(cm_race) # print confusion matrix 

# calculate different totals
[non_recid_race, recid_race] = cm_race.sum(axis = 1)
[tot_afram, tot_cauc] = cm_race.sum(axis = 0)
# assign cells in matrix to variable names I understand
tp_race = cm_race.iloc[1,1]
tn_race = cm_race.iloc[0,0]
fn_race = cm_race.iloc[1,0]
fp_race = cm_race.iloc[0,1]

# print results/answers for accuracy and false rates
# A
print("\nCOMPAS accurately classifies " + str(round(tn_race/tot_afram * 100, 2)) + "% of African Americans")
print("COMPAS accurately classifies " + str(round(tp_race/tot_cauc * 100, 2)) + "% of Caucasians")

# B 
print("\nFalse recidivism rates: " + str(round(fp_race/non_recid_race * 100, 2)) + "%")
print("False no-recidivism rates: " + str(round(fn_race/recid_race * 100, 2)) + "%")

race            African-American  Caucasian
two_year_recid                             
0                           1514       1281
1                           1661        822

COMPAS accurately classifies 47.69% of African Americans
COMPAS accurately classifies 39.09% of Caucasians

False recidivism rates: 45.83%
False no-recidivism rates: 66.89%


#### 8

The COMPAS algorithm is already questionable in the accuracy just based of scoring and recidivism, as it has over 30% error. Hoever, the fact that, based off of race and recidivism, COMPAS cannot even accurately classify at least 50% of races accurately and false positive and false negative rates are even higher than the last confusion matrix, makes me more certain that COMPAS is unfair, particularly racially. Seeing as the recidivism rates for African-Americans is ~ 66.89% and Caucasians is ~ 33.11% and with my prior knowledge from articles of COMPAS giving higher scores to African-Americans of no-recidivism and lower scores to Caucasians of recidivism, the data confirmed the discrepancies of recidivism based off of both score and race. 

## Can you beat COMPAS?  
#### 1

While I like how APRF shows in count accuracy and error, as it is not sensitive to confidence of its estimators (and accuracy is what we care about here), I am the most comfortable with linear regression (or in this case with catagorical variables a logisitic regression), especially because we want to understand the dependence of recidivism rates on many variables/factors. 

#### 2

[Cited formatting for cross_val_score](https://machinelearningmastery.com/repeated-k-fold-cross-validation-with-python/)

In [29]:
# make X and y, X the different catagorical explanatory variables, y is the recidivism binary
# do not include race sex, and numerical version of age because it relates to age_cat

# make dummy with catagory type of prior crimes with inequalities, value is inclusive to right bin
df_compas["prior_cat"] = pd.cut(df_compas.priors_count,
                      bins = [-np.inf, 1, 6, np.inf],
                      labels = ["0", "1-5", "6+"],
                      right=False)

# make X df of explanatory and y of predicting variable
X = df_compas[["age_cat", "c_charge_degree", "prior_cat", "risk_cat"]]
y = df_compas.two_year_recid.array
# have to make X dummy bc not work below if not
X = pd.get_dummies(X)
# get ready cv (cross-validaiton)
cvType = KFold(n_splits=10, shuffle=True, random_state=0)
# assign model to use in cross_val
useModel = LogisticRegression(solver = 'liblinear')
# evaluate model
result = cross_val_score(useModel, X, y, cv = cvType, n_jobs =-1)
print("Percent of correct catagorization: " + str(round(result.mean()*100, 2)) + "% (with standard dev = " + str(round(result.std()*100, 2)) + "%)")

Percent of correct catagorization: 67.19% (with standard dev = 1.68%)


#### 3

[Citation for making interaction effects (commented below)](https://stats.stackexchange.com/questions/105543/how-to-prepare-interactions-of-categorical-variables-in-scikit-learn)

In [30]:
# play around and make new variables for X and test them to see if accuracy increases in predictions

# make prior cat with more bins
df_compas["priors_cat"] = pd.cut(df_compas.priors_count,
                      bins = [-np.inf, 1, 3, 5, 7, np.inf],
                      labels = ["0", "1-2", "3-4", "5-6", "7+"],
                      right=False)

df_compas["age_cat_more"] = pd.cut(df_compas.age,
                      bins = [18, 22, 26, 30, 34, 38, 42, np.inf],
                      labels = ["18-21", "22-25", "26-29", "30-33", "34-37", "38-41", "42+"],
                      right=False)


# from patsy import dmatrices
# did not improve, do not include 
# create dummy variables, and their interactions
#y_1, X_1 = dmatrices('two_year_recid ~ C(c_charge_degree) + C(priors_cat) + C(age_cat_more) + C(risk_cat) + age', df_compas, return_type="dataframe")
# flatten y into a 1-D array so scikit-learn can understand it
#y_1 = np.ravel(y)

# change variable names to 3 to refer to problem 3 analysis
# make X df of explanatory and y of predicting variable
X_3 = df_compas[["age_cat_more", "c_charge_degree", "priors_cat", "risk_cat"]]
y_3 = df_compas.two_year_recid.array
# have to make X_1 dummy bc not work below if not
X_3 = pd.get_dummies(X_3)
# get ready cv (cross-validaiton)
cvType_3 = KFold(n_splits=10, shuffle=True, random_state=0)
# assign model to use in cross_val
useModel_3 = LogisticRegression(solver = 'liblinear')
# evaluate model
result_3 = cross_val_score(useModel_3, X_3, y_3, cv = cvType_3, n_jobs =-1)
print("Percent of correct catagorization: " + str(round(result_3.mean()*100, 2)) + "% (with standard dev = " + str(round(result_3.std()*100, 2)) + "%)")


Percent of correct catagorization: 68.49% (with standard dev = 2.1%)


I attempted various things, some like the suggestions: I increased the number of catagories for the variables, and that increased the accuracy by ~1% (but also increased standard deviation), but any more number of catagories and the accuracy decreased it, and increasing of any other catagoriy levels (like docile score, priors_count, etc.) also decreased the accuracy. Any other attemps of removing 1-2 of the explanatory variables of various combinations and adding interaction effects between different variables, squaring age, etc. all resulted in a decrease in accuracy. I never used age catagory and age together because age catagory was directly related to age, which is a similar issue with removing the score_text variable. 

#### 4

In [31]:
# make X df of explanatory (adding sex) and y of predicting variable
X_4 = df_compas[["age_cat_more", "c_charge_degree", "priors_cat", "risk_cat", "sex"]]
y_4 = df_compas.two_year_recid.array
# have to make X_1 dummy bc not work below if not
X_4 = pd.get_dummies(X_4)
# get ready cv (cross-validaiton)
cvType_4 = KFold(n_splits=10, shuffle=True, random_state=0)
# assign model to use in cross_val
useModel_4 = LogisticRegression(solver = 'liblinear')
# evaluate model
result_4 = cross_val_score(useModel_4, X_4, y_4, cv = cvType_4, n_jobs =-1)
print("Percent of correct catagorization: " + str(round(result_4.mean()*100, 2)) + "% (with standard dev = " + str(round(result_4.std()*100, 2)) + "%)")

Percent of correct catagorization: 68.13% (with standard dev = 1.96%)


The accuracy of the model decreased, but not by much. The decrease in accuracy was followed by a decrease in standard deviation. So in conclusion, the model did not improve, but the worsening effect was not huge.

#### 5

In [32]:
# make X df of explanatory (adding sex and race) and y of predicting variable
X_5 = df_compas[["age_cat_more", "c_charge_degree", "priors_cat", "risk_cat", "sex", "race"]]
y_5 = df_compas.two_year_recid.array
# have to make X_1 dummy bc not work below if not
X_5 = pd.get_dummies(X_5)
# get ready cv (cross-validaiton)
cvType_5 = KFold(n_splits=10, shuffle=True, random_state=0)
# assign model to use in cross_val
useModel_5 = LogisticRegression(solver = 'liblinear')
# evaluate model
result_5 = cross_val_score(useModel_5, X_5, y_5, cv = cvType_5, n_jobs =-1)
print("Percent of correct catagorization: " + str(round(result_5.mean()*100, 2)) + "% (with standard dev = " + str(round(result_5.std()*100, 2)) + "%)")

Percent of correct catagorization: 68.13% (with standard dev = 2.04%)


The accuracy did not change, which intially made me think my code was wrong until I saw the increase in standard deviation. In conclusion, the addition of race does not seem to have a significant affect on the accuracy of the catagorization of this model. 

#### 6

Compared to the accuracy of the confusion matrixes of recidivism vs decile_score and race (65.82% and 44.26% respectively), the accuracy in the last model (68.13%) is larger. So I would say it is a better model than COMPAS. Although there is not much of an increase from recidivism vs docile_score to our model, I had to keep in mind that the comfusion matrix accuracy did not include race nor sex, so it is a significant improvement. Although gender and race decreased the overall accuracy, it was not a large decrease, so I would not say it worsened nor improved my predictions, if anything it kept it about the same. However, realistically, this accuracy is not even close to the ideal percentage I would want judges using. Although the accuracy is now a majority (> 50%) when also considering sex and race, I do not think judges should rely on this heavily for their decisons. While they could use it as a reference, I think there is much more to a person's charge than demographic or scores that would change the act recidivism and should be taken "with a grain of salt."

Hours spent: ~ 6 hours